In [1]:
from src.data_preprocessor import DataProcessor
from src.cross_validation import CrossValidation
from src.evaluation import Evaluation
from models.knn import KNN
from models.null_model import NullModelClassification, NullModelRegression
from data_configs.configs import *

config = car_config
data_processor = DataProcessor(config=config)
cross_validator = CrossValidation(config=config)
classification_nullmodel = NullModelClassification(config=config)
regression_nullmodel = NullModelRegression(config=config)
knn_model = KNN(config)

In [2]:
# Data Processing

raw_data = data_processor.load_data()

data_1 = data_processor.impute_missing_values(raw_data)

# data_2 = data_processor.encode_nominal_features(data_1)

# data_3 = data_processor.encode_ordinal_features(data_2)

In [3]:
data_train, data_val = cross_validator.random_partition(data_1, random_state=42)

In [4]:
data_train

,buying,maint,doors,persons,lug_boot,safety,Class
107,vhigh,vhigh,5more,more,big,high,unacc
901,med,vhigh,3,4,small,med,unacc
1709,low,low,5more,2,big,high,unacc
706,high,med,4,2,med,med,unacc
678,high,med,3,2,med,low,unacc
...,...,...,...,...,...,...,...
1130,med,med,3,more,med,high,vgood
1294,med,low,5more,more,big,med,good
860,high,low,5more,more,med,high,acc
1459,low,high,4,2,small,med,unacc


In [5]:
knn_model.edited_knn_classificaton(data_train)

,buying,maint,doors,persons,lug_boot,safety,Class
107,vhigh,vhigh,5more,more,big,high,unacc
901,med,vhigh,3,4,small,med,unacc
1709,low,low,5more,2,big,high,unacc
706,high,med,4,2,med,med,unacc
678,high,med,3,2,med,low,unacc
...,...,...,...,...,...,...,...
121,vhigh,high,2,4,med,med,unacc
1638,low,low,2,more,small,low,unacc
1044,med,high,4,more,small,low,unacc
1095,med,med,2,4,big,low,unacc


In [6]:
result = knn_model.knn_classifier(data_val,data_train,5)
Evaluation().zero_one_loss(data_val[config['target_column']],result['Predicted Class'])

0.3208092485549133

In [7]:
f1_score = Evaluation().f1_score(data_val[config['target_column']],result['Predicted Class'])

print(f1_score)

0.5494314168316535
